In [6]:
!pip install mlflow

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
from datetime  import datetime  
from datetime import timedelta  
import lightgbm as lgb
from sklearn import preprocessing, metrics
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.lightgbm
import seaborn as sns

In [0]:
np.random.seed(0)

In [0]:
data_folder = '/content/drive/My Drive/Kaggle_M5/00_data/'
submission = pd.read_csv(data_folder + 'sample_submission.csv')

In [0]:
params = {
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'objective':  'tweedie',
        'n_jobs': -1,
        'seed': 0,
        'learning_rate': 0.05, 
        'bagging_fraction': 0.9,
        'bagging_freq': 1, 
        'colsample_bytree': 0.75,
        'num_iterations': 5000,
        'early_stopping_round': 500,
        'max_bin': 100,
        'boost_from_average': False}

In [0]:
params = {'bagging_fraction': 0.5, 
          'bagging_freq': 10, 
          'boosting_type': 'gbdt', 
          'colsample_bytree': 0.75, 
          'early_stopping_round': 500, 
          'learning_rate': 0.05, 
          'metric': 'rmse', 
          'n_jobs': -1, 
          'num_iterations': 5000, 
          'early_stopping_round': 500,         
          'objective': 'tweedie', 
          'seed': 0}

In [0]:
param_grid = {'boosting_type': ['gbdt', 'dart'],
        'metric': ['rmse'],
        'objective': ['poisson', 'tweedie'],
        'n_jobs': [-1],
        'seed': [0],
        'learning_rate':  [0.05, 0.075],
        'bagging_fraction': [0.75],
        'bagging_freq': [5, 10], 
        'colsample_bytree': [0.75],
        'num_iterations': [5000],
        'early_stopping_round': [500]}

In [0]:
evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]

validation_rows = [row for row in submission['id'] if 'validation' in row] 
validation_csv = submission[submission['id'].isin(validation_rows)]


In [0]:
not_features = ['d', 'id', 'item_id', 'store_id', 'demand', 'date', 'start_date']

In [0]:
validation = pd.DataFrame(columns=submission.columns)
data_folder = '/content/drive/My Drive/Kaggle_M5/01_preprocessed_data/'

overall_rmse = 0
pred_true = pd.DataFrame(columns=['pred', 'true'])
rmse_score = []
mae_score = []
j = 0
for param in ParameterGrid(param_grid):
  print("----------------------")
  print(j)
  print(param)
  for i in ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']:
      print(i)
      
      X_val = pd.read_pickle(data_folder + "X_val_{}.pkl".format(i))
      X_test = pd.read_pickle(data_folder + "X_test_{}.pkl".format(i))
      X_train = pd.read_pickle(data_folder + "X_train_{}.pkl".format(i))
      y_train = X_train['demand']
      y_val = X_val['demand']
      print(X_train.shape)
      features = X_train.columns[~X_train.columns.isin(not_features) ]

      train_set = lgb.Dataset(X_train[features],y_train)
      val_set = lgb.Dataset(X_val[features], y_val)
      model = lgb.train(param, train_set, valid_sets = [train_set, val_set], verbose_eval = 100)
      
      val_pred = model.predict(X_val[features])
      val_score = metrics.mean_absolute_error(val_pred, y_val)

      temp = pd.DataFrame({'pred':val_pred, 'true':y_val})
      pred_true =pred_true.append(temp)
      print(f'Our val mae score is {val_score}')

      y_test = model.predict(X_test[features])
      X_test['demand'] = y_test
      predictions = X_test[['id', 'days_from_start', 'demand']]
      predictions = pd.pivot(predictions, index = 'id', columns = 'days_from_start', values = 'demand').reset_index()
      predictions.columns = ['id'] + ['F' + str(j + 1) for j in range(28)]
      validation = validation.append(predictions)
  rmse = np.sqrt(np.mean((pred_true['pred'] - pred_true['true'])**2) )
  rmse_score.append(rmse)
  print("oooooooooooooooooooooooo Overall RMSE oooooooooooooooooooooooo")
  print(rmse_score)
  mae_score.append(np.mean(np.abs(pred_true['pred'] - pred_true['true']) ))
  j = j + 1
    

----------------------
0
{'bagging_fraction': 0.75, 'bagging_freq': 5, 'boosting_type': 'gbdt', 'colsample_bytree': 0.75, 'early_stopping_round': 500, 'learning_rate': 0.05, 'metric': 'rmse', 'n_jobs': -1, 'num_iterations': 5000, 'objective': 'poisson', 'seed': 0}
CA_1
(4510808, 46)


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 500 rounds.
[100]	training's rmse: 2.78533	valid_1's rmse: 2.24759
[200]	training's rmse: 2.71834	valid_1's rmse: 2.20953
[300]	training's rmse: 2.67628	valid_1's rmse: 2.19993
[400]	training's rmse: 2.63692	valid_1's rmse: 2.18155
[500]	training's rmse: 2.60285	valid_1's rmse: 2.17362
[600]	training's rmse: 2.57496	valid_1's rmse: 2.16809
[700]	training's rmse: 2.54864	valid_1's rmse: 2.16504
[800]	training's rmse: 2.5327	valid_1's rmse: 2.16549
[900]	training's rmse: 2.51406	valid_1's rmse: 2.16536
[1000]	training's rmse: 2.49294	valid_1's rmse: 2.16242
[1100]	training's rmse: 2.47869	valid_1's rmse: 2.16177
[1200]	training's rmse: 2.46517	valid_1's rmse: 2.16115
[1300]	training's rmse: 2.45361	valid_1's rmse: 2.16115
[1400]	training's rmse: 2.44082	valid_1's rmse: 2.16693
[1500]	training's rmse: 2.43155	valid_1's rmse: 2.17632
[1600]	training's rmse: 2.42085	valid_1's rmse: 2.17862
[1700]	training's rmse: 2.41047	valid_1's rmse: 2.1

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 500 rounds.
[100]	training's rmse: 2.20559	valid_1's rmse: 2.14394
[200]	training's rmse: 2.16759	valid_1's rmse: 2.11163
[300]	training's rmse: 2.14553	valid_1's rmse: 2.10674
[400]	training's rmse: 2.12555	valid_1's rmse: 2.10376
[500]	training's rmse: 2.10969	valid_1's rmse: 2.09599
[600]	training's rmse: 2.09531	valid_1's rmse: 2.09106
[700]	training's rmse: 2.08366	valid_1's rmse: 2.08964
[800]	training's rmse: 2.07227	valid_1's rmse: 2.08543
[900]	training's rmse: 2.0603	valid_1's rmse: 2.08124
[1000]	training's rmse: 2.05148	valid_1's rmse: 2.07544
[1100]	training's rmse: 2.04272	valid_1's rmse: 2.07138
[1200]	training's rmse: 2.03596	valid_1's rmse: 2.06846
[1300]	training's rmse: 2.02883	valid_1's rmse: 2.068
[1400]	training's rmse: 2.02249	valid_1's rmse: 2.06199
[1500]	training's rmse: 2.0159	valid_1's rmse: 2.05609
[1600]	training's rmse: 2.01002	valid_1's rmse: 2.0544
[1700]	training's rmse: 2.00395	valid_1's rmse: 2.05336

...

Training until validation scores don't improve for 500 rounds.
[100]	training's rmse: 2.74875	valid_1's rmse: 2.13671
[200]	training's rmse: 2.59756	valid_1's rmse: 2.09752
[300]	training's rmse: 2.51161	valid_1's rmse: 2.06684
[400]	training's rmse: 2.45329	valid_1's rmse: 2.04551
[500]	training's rmse: 2.41203	valid_1's rmse: 2.03336
[600]	training's rmse: 2.37534	valid_1's rmse: 2.02877
[700]	training's rmse: 2.34403	valid_1's rmse: 2.02431
[800]	training's rmse: 2.32089	valid_1's rmse: 2.01797
[900]	training's rmse: 2.29906	valid_1's rmse: 2.02106
[1000]	training's rmse: 2.27509	valid_1's rmse: 2.019
[1100]	training's rmse: 2.25628	valid_1's rmse: 2.02141
[1200]	training's rmse: 2.24164	valid_1's rmse: 2.02244
Early stopping, best iteration is:
[799]	training's rmse: 2.32137	valid_1's rmse: 2.01796
Our val mae score is 0.973186267579071
----------------------
3
{'bagging_fraction': 0.75, 'bagging_freq': 5, 'boosting_type': 'gbdt', 'colsample_bytree': 0.75, 'early_stopping_round': 5

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 500 rounds.
[100]	training's rmse: 2.75678	valid_1's rmse: 2.20561
[200]	training's rmse: 2.69336	valid_1's rmse: 2.16226
[300]	training's rmse: 2.64948	valid_1's rmse: 2.15948
[400]	training's rmse: 2.6101	valid_1's rmse: 2.15467
[500]	training's rmse: 2.58824	valid_1's rmse: 2.16156
[600]	training's rmse: 2.57031	valid_1's rmse: 2.16723
[700]	training's rmse: 2.55209	valid_1's rmse: 2.17212
[800]	training's rmse: 2.54115	valid_1's rmse: 2.1738
[900]	training's rmse: 2.52793	valid_1's rmse: 2.1732
Early stopping, best iteration is:
[425]	training's rmse: 2.6059	valid_1's rmse: 2.15424
Our val mae score is 1.1226090787182719
CA_2
(4082924, 46)


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 500 rounds.
[100]	training's rmse: 2.18502	valid_1's rmse: 2.10791
[200]	training's rmse: 2.15176	valid_1's rmse: 2.0969
[300]	training's rmse: 2.13519	valid_1's rmse: 2.09247
[400]	training's rmse: 2.11441	valid_1's rmse: 2.09211
[500]	training's rmse: 2.09934	valid_1's rmse: 2.08787
[600]	training's rmse: 2.08778	valid_1's rmse: 2.08404
[700]	training's rmse: 2.07869	valid_1's rmse: 2.08461
[800]	training's rmse: 2.06934	valid_1's rmse: 2.07258
[900]	training's rmse: 2.06227	valid_1's rmse: 2.07344
[1000]	training's rmse: 2.05389	valid_1's rmse: 2.07188
[1100]	training's rmse: 2.04533	valid_1's rmse: 2.06842
[1200]	training's rmse: 2.0397	valid_1's rmse: 2.06928
[1300]	training's rmse: 2.03346	valid_1's rmse: 2.06738
[1400]	training's rmse: 2.02773	valid_1's rmse: 2.06738
[1500]	training's rmse: 2.02146	valid_1's rmse: 2.06349
[1600]	training's rmse: 2.01577	valid_1's rmse: 2.06118
[1700]	training's rmse: 2.01044	valid_1's rmse: 2.05

In [0]:
np.sqrt(np.mean((pred_true['pred'] - pred_true['true'])**2) )

In [0]:
np.mean(np.abs(pred_true['pred'] - pred_true['true']) )

In [0]:
validation = validation.set_index('id')
validation = validation.reindex(index=validation_csv['id'])
validation = validation.reset_index()

In [0]:
validation

In [0]:
final = pd.concat([validation, evaluation])

In [0]:
submission_folder = '/content/drive/My Drive/Kaggle_M5/04_submissions/'
final.to_csv(submission_folder  + 'submission-by_store.csv', index = False)